<a href="https://colab.research.google.com/github/doyeo-n/-Capstone-8_code/blob/main/%5B8%5D%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%A0%84%EC%B2%98%EB%A6%AC_%EC%BD%94%EB%93%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import os
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from tqdm import tqdm

os.chdir(os.path.abspath(os.path.dirname(__file__)))

df = pd.read_excel('../근접관측소.xlsx')

columns_eng = pd.read_csv('columns_eng.txt',header=None,squeeze=True)
columns_kor = pd.read_csv('columns_kor.txt',header=None,squeeze=True)
columns_eng = [i.lower() for i in columns_eng]

key = 'jPrjYNkvcwQd2Fxv3MuT76uexWMio7m5%2BFgUlrk88LRp29Aq6rm98hJguk%2BOkQ9CRt4ho18rhBim1P4NNcr2lA%3D%3D'

start = datetime(2005,1,1).date()
end = datetime.now().date()-timedelta(1)

numOfRows = '370'
dataCd = 'ASOS'
dateCd = 'DAY'

pbar = tqdm(df['근접관측소코드(기상)'].drop_duplicates())
idx = df['근접관측소코드(기상)'].drop_duplicates().drop_duplicates().index
i = 0


for code in pbar:
    
    current = start
    data = {}
    pbar.set_description(df['근접관측소명(기상)'][idx[i]])
    
    while (current < end):
        
        first = current.strftime('%Y%m%d')
        
        if current.year == 2021:
            last = end.strftime('%Y%m%d')
        else:
            last = (current+relativedelta(years=1)-timedelta(1)).strftime('%Y%m%d')
        
        queryParams = 'servicekey='+key+'&numOfRows='+numOfRows+'&dataCd='+dataCd+'&dateCd='\
        +dateCd+'&startDt='+first+'&endDt='+last+'&stnIds='+str(code)
        
        url = 'http://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList?'+queryParams
        
        req = requests.get(url,allow_redirects=False)
        html = req.text
        soup = BeautifulSoup(html, 'html.parser')
        
        for each in columns_eng:
            finded = soup.find_all(each)
            
            if data.get(each) is None:
                data[each] = [x.text for x in finded]
            else:
                data[each] = data[each] + [x.text for x in finded]
        
        current += relativedelta(years=1)
        
    result = pd.DataFrame(data)
    result.columns = columns_kor
    result.to_excel(df['근접관측소명(기상)'][idx[i]]+'.xlsx',index=None)
    
    i += 1